Установим нужные библиотеки:

In [ ]:
!pip install praat-parselmouth

Скачаем данные:

In [ ]:
!wget pkholyavin.github.io/speechtechhse/vowels01.zip
!unzip vowels01.zip

Вычислим MFCC:

In [30]:
from librosa.feature import mfcc
from librosa import load
import numpy as np

In [19]:
audio, sr = load("/content/vowels/vowels_all/a_andre_00003.wav", sr=None)

In [24]:
mfcc_coeffs = mfcc(y=audio, sr=sr)

In [ ]:
mfcc_coeffs[:, mfcc_coeffs.shape[1] // 2]  # вектор из середины
np.mean(mfcc_coeffs, axis=1) # среднее значение

Вычислим значение первой и второй форманты:

In [32]:
import parselmouth

Функция для вычисления формант в середине звукового файла:

In [33]:
def get_formants(filename):
  try:
    sound = parselmouth.Sound(filename)
  except parselmouth.PraatError:
    return None, None
  formants = sound.to_formant_burg()
  return (
      formants.get_value_at_time(1, formants.centre_time),
      formants.get_value_at_time(2, formants.centre_time)
  )

Импортируем нужные библиотеки:

In [34]:
import glob
from collections import defaultdict

Обработаем все файлы в папке:

In [42]:
data = defaultdict(list)
sounddir = "/content/vowels/vowels_all" # сюда можно подставить другие папки
files = glob.glob(f"{sounddir}/**/*.wav", recursive=True)
for file in files:
    f1, f2 = get_formants(file)
    if f1 is not None:
        data[file.split("/")[-1].split("_")[0]].append((f1, f2))

Нарисуем график:

In [ ]:
import matplotlib.pyplot as plt
for vowel in data:
    plt.scatter(*zip(*data[vowel]), label=vowel)
plt.legend()
plt.show()

Построим классификатор:

In [37]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC
from random import shuffle

In [ ]:
clf = DecisionTreeClassifier() # здесь можно поставить любой другой из перечисленных выше и посмотреть, что получится
training_data = [(el, label) for label in data for el in data[label] if label in ["i", "a", "u"]] # здесь можно перечислить те гласные, которые будем различать (их всего 6: i e a o u y)
shuffle(training_data)
cutoff = int(len(training_data) * 0.9)
training_data, test_data = training_data[:cutoff], training_data[cutoff:]
clf.fit(*zip(*training_data))

Протестируем модель:

In [45]:
from sklearn.metrics import accuracy_score

In [ ]:
test_features, test_labels = zip(*test_data)
pred = clf.predict(test_features)
print(accuracy_score(test_labels, pred))

Если мы использовали дерево, попробуем его визуализировать:

In [ ]:
fig, ax = plt.subplots()
plot_tree(clf)
plt.savefig("tree.pdf")

Эта ячейка - для тестирования модели на своих данных (запишите свой голос и загрузите файл)

In [ ]:
f1, f2 = get_formants("u.wav")
print(f1, f2)

In [ ]:
clf.predict([(f1, f2)])